In [1]:
import pandas as pd
import pandas as pd    #用于数据处理和分析，可处理表格数据。
import numpy as np     #用于数值计算，提供了高效的数组操作。
import matplotlib.pyplot as plt    #用于绘制各种类型的图表
import seaborn as sns   #基于matplotlib的高级绘图库，能绘制更美观的统计图形。
import warnings
warnings.filterwarnings("ignore")

# 设置中文字体（解决中文显示问题）
plt.rcParams['font.sans-serif'] = ['SimHei']  # Windows系统常用黑体字体
plt.rcParams['axes.unicode_minus'] = False    # 正常显示负号
data = pd.read_csv('data.csv')    #读取数据

# 先筛选字符串变量 
discrete_features = data.select_dtypes(include=['object']).columns.tolist()
# Home Ownership 标签编码
home_ownership_mapping = {
    'Own Home': 1,
    'Rent': 2,
    'Have Mortgage': 3,
    'Home Mortgage': 4
}
data['Home Ownership'] = data['Home Ownership'].map(home_ownership_mapping)

# Years in current job 标签编码
years_in_job_mapping = {
    '< 1 year': 1,
    '1 year': 2,
    '2 years': 3,
    '3 years': 4,
    '4 years': 5,
    '5 years': 6,
    '6 years': 7,
    '7 years': 8,
    '8 years': 9,
    '9 years': 10,
    '10+ years': 11
}
data['Years in current job'] = data['Years in current job'].map(years_in_job_mapping)

# Purpose 独热编码，记得需要将bool类型转换为数值
data = pd.get_dummies(data, columns=['Purpose'])
data2 = pd.read_csv("data.csv") # 重新读取数据，用来做列名对比
list_final = [] # 新建一个空列表，用于存放独热编码后新增的特征名
for i in data.columns:
    if i not in data2.columns:
       list_final.append(i) # 这里打印出来的就是独热编码后的特征名
for i in list_final:
    data[i] = data[i].astype(int) # 这里的i就是独热编码后的特征名

# Term 0 - 1 映射
term_mapping = {
    'Short Term': 0,
    'Long Term': 1
}
data['Term'] = data['Term'].map(term_mapping)
data.rename(columns={'Term': 'Long Term'}, inplace=True) # 重命名列
continuous_features = data.select_dtypes(include=['int64', 'float64']).columns.tolist()  #把筛选出来的列名转换成列表
 
# 连续特征用中位数补全
for feature in continuous_features:     
    mode_value = data[feature].mode()[0]            #获取该列的众数。
    data[feature].fillna(mode_value, inplace=True)          #用众数填充该列的缺失值，inplace=True表示直接在原数据上修改。
for feature in continuous_features:     
    mode_value = data[feature].mode()[0]            #获取该列的众数。
    data[feature].fillna(mode_value, inplace=True)          #用众数填充该列的缺失值，inplace=True表示直接在原数据上修改。

# 最开始也说了 很多调参函数自带交叉验证，甚至是必选的参数，你如果想要不交叉反而实现起来会麻烦很多
# 所以这里我们还是只划分一次数据集
from sklearn.model_selection import train_test_split
X = data.drop(['Credit Default'], axis=1)  # 特征，axis=1表示按列删除
y = data['Credit Default'] # 标签
# 按照8:2划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # 80%训练集，20%测试集



c:\Anaconda\envs\vs\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:

from sklearn.ensemble import RandomForestClassifier #随机森林分类器

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # 用于评估分类器性能的指标
from sklearn.metrics import classification_report, confusion_matrix #用于生成分类报告和混淆矩阵
import warnings #用于忽略警告信息
warnings.filterwarnings("ignore") # 忽略所有警告信息
# --- 1. 默认参数的随机森林 ---
# 评估基准模型，这里确实不需要验证集
print("--- 1. 默认参数随机森林 (训练集 -> 测试集) ---")
import time # 这里介绍一个新的库，time库，主要用于时间相关的操作，因为调参需要很长时间，记录下会帮助后人知道大概的时长
start_time = time.time() # 记录开始时间
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train) # 在训练集上训练
rf_pred = rf_model.predict(X_test) # 在测试集上预测
end_time = time.time() # 记录结束时间

print(f"训练与预测耗时: {end_time - start_time:.4f} 秒")
print("\n默认随机森林 在测试集上的分类报告：")
print(classification_report(y_test, rf_pred))
print("默认随机森林 在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred))

--- 1. 默认参数随机森林 (训练集 -> 测试集) ---
训练与预测耗时: 0.8057 秒

默认随机森林 在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.77      0.97      0.86      1059
           1       0.79      0.30      0.43       441

    accuracy                           0.77      1500
   macro avg       0.78      0.63      0.64      1500
weighted avg       0.77      0.77      0.73      1500

默认随机森林 在测试集上的混淆矩阵：
[[1023   36]
 [ 309  132]]


In [3]:
# 以下是添加的过采样代码

# 1. 随机过采样
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42) # 创建随机过采样对象
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train) # 对训练集进行随机过采样

print("随机过采样后训练集的形状：", X_train_ros.shape, y_train_ros.shape) 

# 训练随机森林模型（使用随机过采样后的训练集）
rf_model_ros = RandomForestClassifier(random_state=42)
start_time_ros = time.time()
rf_model_ros.fit(X_train_ros, y_train_ros)
end_time_ros = time.time()

print(f"随机过采样后训练与预测耗时: {end_time_ros - start_time_ros:.4f} 秒")

# 在测试集上预测
rf_pred_ros = rf_model_ros.predict(X_test)

print("\n随机过采样后随机森林 在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_ros))
print("随机过采样后随机森林 在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_ros))

随机过采样后训练集的形状： (8656, 31) (8656,)
随机过采样后训练与预测耗时: 1.0203 秒

随机过采样后随机森林 在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.77      0.93      0.84      1059
           1       0.67      0.34      0.46       441

    accuracy                           0.76      1500
   macro avg       0.72      0.64      0.65      1500
weighted avg       0.74      0.76      0.73      1500

随机过采样后随机森林 在测试集上的混淆矩阵：
[[985  74]
 [289 152]]


In [4]:
# 2. SMOTE过采样
from imblearn.over_sampling import SMOTE 
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("SMOTE过采样后训练集的形状：", X_train_smote.shape, y_train_smote.shape)

# 训练随机森林模型（使用SMOTE过采样后的训练集）
rf_model_smote = RandomForestClassifier(random_state=42)
start_time_smote = time.time()
rf_model_smote.fit(X_train_smote, y_train_smote)
end_time_smote = time.time()

print(f"SMOTE过采样后训练与预测耗时: {end_time_smote - start_time_smote:.4f} 秒")

# 在测试集上预测
rf_pred_smote = rf_model_smote.predict(X_test)

print("\nSMOTE过采样后随机森林 在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_smote))
print("SMOTE过采样后随机森林 在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_smote))

SMOTE过采样后训练集的形状： (8656, 31) (8656,)
SMOTE过采样后训练与预测耗时: 1.0792 秒

SMOTE过采样后随机森林 在测试集上的分类报告：
              precision    recall  f1-score   support

           0       0.77      0.92      0.84      1059
           1       0.64      0.35      0.45       441

    accuracy                           0.75      1500
   macro avg       0.70      0.63      0.64      1500
weighted avg       0.73      0.75      0.72      1500

SMOTE过采样后随机森林 在测试集上的混淆矩阵：
[[972  87]
 [288 153]]


In [ ]:
import numpy as np # 引入 numpy 用于计算平均值等
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate # 引入分层 K 折和交叉验证工具
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import time
import warnings
warnings.filterwarnings("ignore")

# 假设 X_train, y_train, X_test, y_test 已经准备好
# X_train, y_train 用于交叉验证和最终模型训练
# X_test, y_test 用于最终评估

# --- 1. 默认参数的随机森林 (原始代码，作为对比基准) ---
print("--- 1. 默认参数随机森林 (训练集 -> 测试集) ---")
start_time = time.time()
rf_model_default = RandomForestClassifier(random_state=42)
rf_model_default.fit(X_train, y_train)
rf_pred_default = rf_model_default.predict(X_test)
end_time = time.time()
print(f"默认模型训练与预测耗时: {end_time - start_time:.4f} 秒")
print("\n默认随机森林 在测试集上的分类报告：")
print(classification_report(y_test, rf_pred_default))
print("默认随机森林 在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_default))
print("-" * 50)


# --- 2. 带权重的随机森林 + 交叉验证 (在训练集上进行CV) ---
print("--- 2. 带权重随机森林 + 交叉验证 (在训练集上进行) ---")

# 确定少数类标签 (非常重要！)
# 假设是二分类问题，我们需要知道哪个是少数类标签才能正确解读 recall, precision, f1
# 例如，如果标签是 0 和 1，可以这样查看：
counts = np.bincount(y_train)
minority_label = np.argmin(counts) # 找到计数最少的类别的标签
majority_label = np.argmax(counts)
print(f"训练集中各类别数量: {counts}")
print(f"少数类标签: {minority_label}, 多数类标签: {majority_label}")
# !!下面的 scorer 将使用这个 minority_label !!

# 定义带权重的模型
rf_model_weighted = RandomForestClassifier(
    random_state=42,
    class_weight='balanced'  # 关键：自动根据类别频率调整权重
    # class_weight={minority_label: 10, majority_label: 1} # 或者可以手动设置权重字典
)

# 设置交叉验证策略 (使用 StratifiedKFold 保证每折类别比例相似)
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # 5折交叉验证

# 定义用于交叉验证的评估指标
# 特别关注少数类的指标，使用 make_scorer 指定 pos_label
# 注意：如果你的少数类标签不是 1，需要修改 pos_label
scoring = {
    'accuracy': 'accuracy',
    'precision_minority': make_scorer(precision_score, pos_label=minority_label, zero_division=0),
    'recall_minority': make_scorer(recall_score, pos_label=minority_label),
    'f1_minority': make_scorer(f1_score, pos_label=minority_label)
}

print(f"开始进行 {cv_strategy.get_n_splits()} 折交叉验证...")
start_time_cv = time.time()

# 执行交叉验证 (在 X_train, y_train 上进行)
# cross_validate 会自动完成训练和评估过程
cv_results = cross_validate(
    estimator=rf_model_weighted,
    X=X_train,
    y=y_train,
    cv=cv_strategy,
    scoring=scoring,
    n_jobs=-1, # 使用所有可用的 CPU 核心
    return_train_score=False # 通常我们更关心测试折的得分
)

end_time_cv = time.time()
print(f"交叉验证耗时: {end_time_cv - start_time_cv:.4f} 秒")

# 打印交叉验证结果的平均值
print("\n带权重随机森林 交叉验证平均性能 (基于训练集划分)：")
for metric_name, scores in cv_results.items():
    if metric_name.startswith('test_'): # 我们关心的是在验证折上的表现
         # 提取指标名称（去掉 'test_' 前缀）
        clean_metric_name = metric_name.split('test_')[1]
        print(f"  平均 {clean_metric_name}: {np.mean(scores):.4f} (+/- {np.std(scores):.4f})")

print("-" * 50)


# --- 3. 使用权重训练最终模型，并在测试集上评估 ---
print("--- 3. 训练最终的带权重模型 (整个训练集) 并在测试集上评估 ---")
start_time_final = time.time()
# 使用与交叉验证中相同的设置来训练最终模型
rf_model_weighted_final = RandomForestClassifier(
    random_state=42,
    class_weight='balanced'
)
rf_model_weighted_final.fit(X_train, y_train) # 在整个训练集上训练
rf_pred_weighted = rf_model_weighted_final.predict(X_test) # 在测试集上预测
end_time_final = time.time()

print(f"最终带权重模型训练与预测耗时: {end_time_final - start_time_final:.4f} 秒")
print("\n带权重随机森林 在测试集上的分类报告：")
# 确保 classification_report 也关注少数类 (可以通过 target_names 参数指定标签名称)
# 或者直接查看报告中少数类标签对应的行
print(classification_report(y_test, rf_pred_weighted)) # , target_names=[f'Class {majority_label}', f'Class {minority_label}'] 如果需要指定名称
print("带权重随机森林 在测试集上的混淆矩阵：")
print(confusion_matrix(y_test, rf_pred_weighted))
print("-" * 50)

# 对比总结 (简单示例)
print("性能对比 (测试集上的少数类召回率 Recall):")
recall_default = recall_score(y_test, rf_pred_default, pos_label=minority_label)
recall_weighted = recall_score(y_test, rf_pred_weighted, pos_label=minority_label)
print(f"  默认模型: {recall_default:.4f}")
print(f"  带权重模型: {recall_weighted:.4f}")

